In [8]:
import os
from ultralytics import YOLO
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import numpy as np

# 1. YOLO 모델 로드
yolo_model = YOLO(r"C:\Users\windows11\Downloads\CBR LNP.v1i.yolov8\YOLOv8_training\experiment19\weights\best.pt")


# 2. 이미지 폴더에서 파일 읽기
image_folder = r"C:\Users\windows11\Downloads\1"  # 이미지가 저장된 폴더 경로
labels = []  # 각 이미지의 장면 유형 라벨
detected_objects_list = []  # YOLO 탐지 결과

# 폴더 내 모든 이미지 파일 읽기
for idx, file_name in enumerate(os.listdir(image_folder)):
    if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):  # 이미지 파일만 처리
        image_path = os.path.join(image_folder, file_name)
        print(f"Processing image {idx + 1}: {file_name}")

        # 이미지 로드 및 YOLO 탐지 수행
        image = Image.open(image_path)
        results = yolo_model(image)

        # 탐지된 객체 이름 추출
        detected_objects = []
        for result in results:
            if result.boxes is not None:
                for box in result.boxes:
                    cls = int(box.cls.item())
                    class_name = yolo_model.names[cls]
                    detected_objects.append(class_name)

        # 탐지 결과를 리스트에 추가
        if detected_objects:
            detected_objects_list.append(" ".join(detected_objects))
        else:
            detected_objects_list.append("")  # 빈 텍스트 추가 (탐지되지 않은 경우)

        # 이미지 라벨 수동 추가 (예제)
        # 이미지 이름을 기반으로 라벨 결정 (필요시 수정 가능)
        if "chem" in file_name.lower():
            labels.append("chemistry_lab")
        elif "bio" in file_name.lower():
            labels.append("biology_lab")
        elif "rad" in file_name.lower():
            labels.append("radioactive_lab")
        else:
            labels.append("unknown_lab")

# 3. 데이터 확인
print(f"Detected objects list: {detected_objects_list}")
print(f"Labels: {labels}")

# 데이터 길이 확인 및 동기화
if len(detected_objects_list) != len(labels):
    print("Mismatch detected. Adjusting detected_objects_list or labels...")
    while len(detected_objects_list) < len(labels):
        detected_objects_list.append("")
    if len(detected_objects_list) > len(labels):
        detected_objects_list = detected_objects_list[:len(labels)]

print(f"Adjusted lengths - Detected objects: {len(detected_objects_list)}, Labels: {len(labels)}")

# **LabelEncoder 적용 (여기에서 변환)**
print("Encoding labels...")
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)
print(f"Encoded Labels: {y_encoded}")
print(f"Label Mapping: {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")

# 4. TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(detected_objects_list)
print(f"TF-IDF Shape: {X.shape}")


# 5. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
print(f"Train Data Shape: {X_train.shape}, Test Data Shape: {X_test.shape}")


# 6. XGBoost 모델 학습
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
print("XGBoost Training Complete.")

# 7. 모델 검증
y_pred = xgb_model.predict(X_test)

y_pred_decoded = label_encoder.inverse_transform(y_pred)  # 디코딩
y_test_decoded = label_encoder.inverse_transform(y_test)  # 디코딩
print("Classification Report:")
print(classification_report(y_test, y_pred))


Processing image 1: bio_lab1.jpg



c:\Users\windows11\anaconda3\envs\yolov8\Lib\site-packages\ultralytics\nn\tasks.py:634: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location="c

0: 320x640 (no detections), 29.9ms
Speed: 8.1ms preprocess, 29.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
Processing image 2: bio_lab_2.jpg

0: 448x640 1 microscope, 34.8ms
Speed: 8.6ms preprocess, 34.8ms inference, 6.8ms postprocess per image at shape (1, 3, 448, 640)
Processing image 3: bio_lab_3.jpg

0: 448x640 (no detections), 39.5ms
Speed: 9.3ms preprocess, 39.5ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)
Processing image 4: chem_lab1.jpg

0: 448x640 3 lab_animals, 38.5ms
Speed: 17.2ms preprocess, 38.5ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)
Processing image 5: chem_lab3.jpg

0: 320x640 (no detections), 28.0ms
Speed: 0.7ms preprocess, 28.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
Processing image 6: chem_lab_2.jpg

0: 288x640 (no detections), 22.2ms
Speed: 1.6ms preprocess, 22.2ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 640)
Processing image 7: rad_lab_1.jpg

0

c:\Users\windows11\anaconda3\envs\yolov8\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\windows11\anaconda3\envs\yolov8\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\windows11\anaconda3\envs\yolov8\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import cv2
from ultralytics import YOLO
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# YOLO 모델 로드
yolo_model = YOLO("path_to_yolo_model/best.pt")

# XGBoost 모델 로드
xgb_model = XGBClassifier()
xgb_model.load_model("path_to_xgb_model.json")

# TF-IDF 벡터화 모델 로드
vectorizer = TfidfVectorizer()
vectorizer = vectorizer.fit(["example_class1 example_class2"])  # 학습된 TF-IDF 모델

# 실시간 객체 탐지 및 분류 함수
def classify_lab_frame(frame):
    # YOLO 탐지 수행
    results = yolo_model(frame)
    detected_objects = []

    # 탐지된 객체 이름 추출
    for result in results:
        if result.boxes is not None:
            for box in result.boxes:
                cls = int(box.cls.item())
                class_name = yolo_model.names[cls]
                detected_objects.append(class_name)

    # 탐지된 객체를 TF-IDF 벡터로 변환
    detected_objects_str = " ".join(detected_objects)
    tfidf_vector = vectorizer.transform([detected_objects_str]).toarray()

    # XGBoost 모델로 분류
    predicted_label = xgb_model.predict(tfidf_vector)[0]
    return predicted_label

# 실시간 카메라 스트림
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # 실험실 분류
    lab_type = classify_lab_frame(frame)
    print(f"Detected Lab Type: {lab_type}")

    # 화면 출력
    cv2.putText(frame, f"Lab Type: {lab_type}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Lab Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
